In [ ]:
# @title Run to download the required libraries and models
print("Downloading and installing libraries...")
%pip install ultralytics supervision roboflow ipywidgets pillow > /dev/null 2>&1
print("Done")
print("Downloading the model...")
!pip install gdown > /dev/null 2>&1
import gdown
file_id = '12q1xnr7_Enz_BwfP8G0DtAM4Z_i2KYhi'
destination = 'yolo11.pt'
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=True)
print("Done")
print("Setup complete!")


In [ ]:
# @title Run the model
import ipywidgets as widgets
from IPython.display import display, Image
from ultralytics import YOLO
import os
from google.colab import files
import torch
from PIL import Image as PILImage
import requests

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO('yolo11.pt').to(device)

def generate_new_filename(base_name, ext):
    count = 1
    new_name = f"{base_name}{ext}"
    while os.path.exists(f"/content/{new_name}"):
        count += 1
        new_name = f"{base_name} ({count}){ext}"
    return new_name

def predict_image(image_path, conf_threshold=0.25):
    try:
        image = PILImage.open(image_path)
        model.predict(image, conf=conf_threshold, save=False)
        results = model.predict(image, conf=conf_threshold, save=True)
        output_path = results[0].save_dir
        image_name = os.listdir(output_path)[0]
        base_name, ext = os.path.splitext(image_name)
        final_image_name = generate_new_filename(base_name, ext)
        final_image_path = f"/content/{final_image_name}"
        os.rename(os.path.join(output_path, image_name), final_image_path)
        return final_image_path
    except Exception as e:
        print(f"An error occurred while processing the image: {str(e)}")
        return None

def predict_video(video_path, conf_threshold=0.25):
    try:
        results = model.predict(source=video_path, conf=conf_threshold, save=True)
        output_path = results[0].save_dir
        video_files = [f for f in os.listdir(output_path) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        if video_files:
            final_video_path = os.path.join(output_path, video_files[0])
            return final_video_path
        else:
            print("No video file found in the output directory.")
            return None
    except Exception as e:
        print(f"An error occurred while processing the video: {str(e)}")
        return None

def download_file_from_url(url, save_path):
    try:
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024):
                    f.write(chunk)
            return save_path
        else:
            print(f"Failed to download file. Status code: {r.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred while downloading the file: {str(e)}")
        return None

input_option = widgets.Dropdown(
    options=['Choose input method', 'Upload File', 'URL', 'File Path'],
    value='Choose input method',
    description='Input Method:',
    disabled=False,
)

upload_widget = widgets.FileUpload(accept='.jpg,.jpeg,.png,.bmp,.mp4,.avi,.mov,.mkv', multiple=False)
url_widget = widgets.Text(value='', placeholder='Enter URL', description='File URL:', disabled=False)
path_widget = widgets.Text(value='', placeholder='Enter file path', description='File Path:', disabled=False)

upload_widget.layout.display = 'none'
url_widget.layout.display = 'none'
path_widget.layout.display = 'none'

conf_slider = widgets.FloatSlider(value=0.25, min=0.1, max=1.0, step=0.05, description='Confidence:')
run_button = widgets.Button(description="Run YOLO")
download_button = widgets.Button(description="Download Result")
output = widgets.Output()

download_button.layout.visibility = 'hidden'

current_output_path = None

def handle_input_change(change):
    if change['new'] == 'Upload File':
        upload_widget.layout.display = 'block'
        url_widget.layout.display = 'none'
        path_widget.layout.display = 'none'
    elif change['new'] == 'URL':
        upload_widget.layout.display = 'none'
        url_widget.layout.display = 'block'
        path_widget.layout.display = 'none'
    elif change['new'] == 'File Path':
        upload_widget.layout.display = 'none'
        url_widget.layout.display = 'none'
        path_widget.layout.display = 'block'
    else:
        upload_widget.layout.display = 'none'
        url_widget.layout.display = 'none'
        path_widget.layout.display = 'none'

input_option.observe(handle_input_change, names='value')

def on_button_click(b):
    global current_output_path
    with output:
        output.clear_output()
        print("Processing...")

        if input_option.value == 'Upload File':
            if upload_widget.value:
                uploaded_file_name = list(upload_widget.value.keys())[0]
                file_data = upload_widget.value[uploaded_file_name]['content']
                with open(uploaded_file_name, 'wb') as f:
                    f.write(file_data)
                file_name = uploaded_file_name
            else:
                print("Please upload a file.")
                return
        elif input_option.value == 'URL':
            url = url_widget.value
            if url:
                file_name = url.split("/")[-1]
                file_name = download_file_from_url(url, f"/content/{file_name}")
                if not file_name:
                    print("Failed to download file from URL.")
                    return
            else:
                print("Please enter a valid URL.")
                return
        elif input_option.value == 'File Path':
            file_name = path_widget.value
            if not os.path.exists(file_name):
                print("File not found at the specified path.")
                return
        else:
            print("Please choose an input method.")
            return

        file_ext = os.path.splitext(file_name)[1].lower()
        is_image = file_ext in ['.jpg', '.jpeg', '.png', '.bmp']
        is_video = file_ext in ['.mp4', '.avi', '.mov', '.mkv']

        if is_image:
            output_image_path = predict_image(file_name, conf_slider.value)
        elif is_video:
            output_image_path = predict_video(file_name, conf_slider.value)
        else:
            print("Unsupported file format. Please upload an image or video.")
            return

        if output_image_path:
            print(f"Done! Result saved as {output_image_path}")
            if is_image:
                display(Image(filename=output_image_path))
            current_output_path = output_image_path
            download_button.layout.visibility = 'visible'
        else:
            print("Failed to process the file.")

def on_download_click(b):
    if current_output_path and os.path.exists(current_output_path):
        print(f"Downloading {current_output_path}...")
        files.download(current_output_path)
    else:
        print("No file to download.")

run_button.on_click(on_button_click)
download_button.on_click(on_download_click)

display(input_option, upload_widget, url_widget, path_widget, conf_slider, run_button, output, download_button)
